In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import jsonlines
import pandas as pd
import numpy as np
#import seaborn as sns
from matplotlib import pyplot as plt
import spacy
import string
from many_stop_words import get_stop_words
nlp = spacy.load('en_core_web_lg')

In [2]:
# punctuation includes the symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
punctuation = string.punctuation
# stopwords are words of lesser interesst like 'and, for, no, because, nobody,...'
stopwords = get_stop_words('en')#list(STOP_WORDS)
special_characters = list("""!"#$%&\()*+,-./:;<=>?@[\\]^_{|}~""")

def filter_string(string):
    # filter out punctuation, digits and stopwords
    filtered_string = ''.join(filter(lambda char: char not in special_characters ,string))
    words = filtered_string.split()
    filtered_words = [word for word in words if word not in stopwords]
    filtered_string = ' '.join(filtered_words)
    #filtered_string = ''.join([char for char in filtered_string if char not in punctuation and not char.isdigit()])
    return filtered_string

<>:5: SyntaxWarning: invalid escape sequence '\('
<>:5: SyntaxWarning: invalid escape sequence '\('
C:\Users\Michael Pasch\AppData\Local\Temp\ipykernel_24984\495593237.py:5: SyntaxWarning: invalid escape sequence '\('
  special_characters = list("""!"#$%&\()*+,-./:;<=>?@[\\]^_{|}~""")


In [3]:
def preprocess_df(df):
    # review to lowercase, filter review, sort by rating and re-index
    df['review'] = df['review'].apply(lambda x: filter_string(str(x).lower()))
    # remove empty columns
    df2 = df[df['review'] != '']
    df2 = df2.sort_values(by='rating')
    df2.index = np.arange(0, len(df2))
    return df2

In [6]:
list_of_vars = ["total_game_hours_last_two_weeks", "num_groups", "num_badges", "num_found_funny", "review", "num_workshop_items", "found_helpful_percentage", "num_voted_helpfulness", "num_achievements_percentage", "num_achievements_attained", "num_achievements_possible", "num_found_helpful", "friend_player_level", "num_found_unhelpful", "total_game_hours", "num_guides", "rating", "num_friends", "num_screenshots", "num_comments", "num_reviews", "num_games_owned"]
list_of_interesting_vars = ["found_helpful_percentage", "review", "num_achievements_percentage",  "num_achievements_possible", "total_game_hours", "rating"]
list_of_games = ['Arma_3', 'Counter_Strike', 'Counter_Strike_Global_Offensive', 'Dota_2', 'Football_Manager_2015', 'Garrys_Mod', 'Grand_Theft_auto_V', 'Sid_Meiers_Civilization_5', 'Team_Fortress_2', 'The_Elder_Scrolls_V', 'Warframe']
data_of_games = dict()

for game in list_of_games:
    data_of_games[game] = pd.read_json('../../dat/' + game + '.jsonlines', lines=True)

In [20]:
def visualize_review_corrcoef(df, title='corrCoefPlot'):
    # embedd the reviews, build corrcoef matrix and visualize it
    df_copy = df.copy(deep=True) # copy or original dataset will be embedded
    df_copy['review'] = df_copy['review'].apply(lambda x: nlp(x).vector)
    np_data = df_copy['review'].to_numpy()
    edge_negative_reviews = len(df_copy[df_copy['rating'] == 'Not Recommended'])
    stacked_data = np_data[0]
    for row in range(np_data.shape[0]):
        stacked_data = np.vstack((stacked_data,np_data[row]))
    corrcoef_matrix = np.corrcoef(stacked_data)
    corrcoef_matrix = np.nan_to_num(corrcoef_matrix)
    # insert white line to separate negative and positive reviews
    for i in range(20):
        corrcoef_matrix = np.insert(corrcoef_matrix, edge_negative_reviews+i, 1, axis=0)
        corrcoef_matrix = np.insert(corrcoef_matrix, edge_negative_reviews, 1, axis=1)
    # set plotsize so that each pixel is one entry in matrix
    plt.figure(figsize=(corrcoef_matrix.shape[0]/100,corrcoef_matrix.shape[1]/100))
    ax = plt.imshow(corrcoef_matrix, cmap=plt.cm.ocean)
    plt.colorbar()
    plt.savefig(f"{title.replace(":", "_")}_matrix.pdf")
    plt.rc('xtick', labelsize=80)
    plt.rc('ytick', labelsize=80)
    plt.title(title, fontisze=80)
    plt.show

In [19]:
# ausführen auf eigene gefahr, da sehr lange laufzeit!
for game in list_of_games:
    title = game
    df_sample = data_of_games[game]#.sample(frac=0.25, replace=False, random_state=1)
    df_sample = preprocess_df(df_sample)
    visualize_review_corrcoef(df_sample, title)

KeyboardInterrupt: 